In [1]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import os
df = pd.read_csv("servicenow_cleaned.csv")

In [3]:
rows_per_group = 100
df_sample = (
    df.groupby('assignment_group',group_keys=False)
    .apply(lambda x: x.sample(min(len(x),rows_per_group),random_state=42))
    .reset_index(drop=True)
)
print("sample created with shape:",df_sample.shape)
df_sample['assignment_group'].value_counts().head()

sample created with shape: (2312, 23)


C:\Users\nehat\AppData\Local\Temp\ipykernel_12472\2577086139.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(min(len(x),rows_per_group),random_state=42))


assignment_group
HCL-AVD-Citrix Support    100
HCL-GSD-English           100
HCL-RHYTHM                100
HCL-SCCM                  100
HCL-Service Now           100
Name: count, dtype: int64

In [4]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [5]:
text_data = df_sample[['incident_number','summary','description','resolution_notes']]

In [7]:
os.makedirs("checkpoints",exist_ok=True)
checkpoints_path = "checkpoints/servicenow_ai_summary.csv"
if 'ai_summary' not in df_sample.columns:
    df_sample['ai_summary'] = None
# Summarization function
def generate_summary(row):
    text = (str(row['summary']) + " " + str(row['description'])+ " " + str(row['resolution_notes'])).strip()
    if len(text) < 50:
        return text  # skip short text
    try:
        result = summarizer(text, max_length=50, min_length=20, do_sample=False)
        return result[0]['summary_text']
    except Exception as e:
        return f"Error: {e}"

# Progress with auto-save
tqdm.pandas()
for i in tqdm(range(len(df_sample))):
    if pd.isnull(df_sample.loc[i, 'ai_summary']):
        df_sample.loc[i, 'ai_summary'] = generate_summary(df_sample.loc[i])
    
    # Save checkpoint every 200 rows
    if i % 200 == 0 and i != 0:
        df_sample.to_csv(checkpoints_path, index=False)
        print(f"✅ Checkpoint saved at row {i}")

# Save final output
df_sample.to_excel("servicenow_ai_summary_final.xlsx", index=False)
print("🎉 Summarization completed and file saved!")

  9%|██████▊                                                                       | 201/2312 [00:00<00:04, 463.39it/s]

✅ Checkpoint saved at row 200


 17%|█████████████▏                                                              | 401/2312 [50:30<11:05:27, 20.89s/it]Your max_length is set to 50, but your input_length is only 34. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)


✅ Checkpoint saved at row 400


 26%|███████████████████▍                                                       | 601/2312 [1:26:57<3:30:43,  7.39s/it]

...

✅ Checkpoint saved at row 2000


 95%|████████████████████████████████████████████████████████████████████████▎   | 2201/2312 [6:08:20<10:25,  5.63s/it]

✅ Checkpoint saved at row 2200


100%|████████████████████████████████████████████████████████████████████████████| 2312/2312 [6:27:54<00:00, 10.07s/it]


🎉 Summarization completed and file saved!
